In [303]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
import sklearn

In [304]:
data = pd.read_csv('../data/手机价格预测.csv')
(x,y) = data.iloc[:,:-1],data.iloc[:,-1]
x.shape,y.shape

((2000, 20), (2000,))

In [305]:
 #数据集划分
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2)
train_x = train_x.astype('float32')
train_y = train_y.astype('int64')
test_x = test_x.astype('float32')
test_y = test_y.astype('int64')
    #标准化
ss = StandardScaler()
train_x = ss.fit_transform(train_x)
test_x = ss.fit_transform(test_x)

#构建循环模型 这里记得要从pd中values出来
train_db = TensorDataset(torch.from_numpy(train_x),torch.tensor(train_y.values))
test_db = TensorDataset(torch.from_numpy(test_x),torch.tensor(test_y.values))

train_db = DataLoader(train_db,shuffle=True,batch_size=8)
test_db = DataLoader(test_db,shuffle=True,batch_size=8)



In [306]:
#网络层定义
class NetWork(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NetWork, self).__init__()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        self.fc1 = nn.Linear(in_features=input_dim,out_features=256)
        self.fc2 = nn.Linear(in_features=256,out_features=512)
        self.fc3 = nn.Linear(in_features=512,out_features=256)
        self.fc4 = nn.Linear(in_features=256,out_features=128)
        self.fc5 = nn.Linear(in_features=128,out_features=output_dim)

        self.dropout = nn.Dropout(0.3)
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [307]:
model = NetWork(train_x.shape[1],np.unique(train_y).shape[0])
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optim = torch.optim.Adam(model.parameters(),lr=0.0001)

# 固定随机数种子
torch.manual_seed(0)

In [308]:
for epoch in range(10):
    for step, (batch_x, batch_y) in enumerate(train_db):
        output = model(batch_x)
        loss = criterion(output,batch_y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    print(epoch,loss.item())

0 1.2277491092681885
1 0.47281894087791443
2 0.3069364130496979
3 0.3556777238845825
4 0.29876047372817993
5 0.8077912330627441
6 0.6989684104919434
7 0.22623923420906067
8 0.5951210260391235
9 0.06958683580160141
